### 버스 노선별 정류장수, 거리, 굴곡도

In [14]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [15]:
busroute_usage = pd.read_csv(
    filepath_or_buffer="../data/gyeongn_busroute_total.csv"
    )

### 웹 크롤링

In [24]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from tqdm import tqdm

In [31]:
city_name = "사천시" 
input_values = busroute_usage[busroute_usage["city_name"] == city_name][["routeno", "startnodenm"]] 
findtime = 100000
route_inform_data = {}


if city_name == "창원시":
    region_button_selector = "48123_48127_48125_48120_48121_48129"
    
elif city_name == "진주시":
    region_button_selector = "48170"
    
elif city_name == "통영시":
    region_button_selector = "48220"
    
elif city_name == "사천시":
    region_button_selector = "48240"
    
elif city_name == "김해시":
    region_button_selector = "48250"
    
elif city_name == "밀양시":
    region_button_selector = "48270"
        
elif city_name == "거제시":
    region_button_selector = "48310"
    
elif city_name == "양산시":
    region_button_selector = "48330"
    
elif city_name == "의령군":
    region_button_selector = "48720"
    
elif city_name == "함안군":
    region_button_selector = "48730"
    
elif city_name == "창녕군":
    region_button_selector = "48740"
    
elif city_name == "고성군":
    region_button_selector = "48820"
    
elif city_name == "남해군":
    region_button_selector = "48840"
    
elif city_name == "하동군":
    region_button_selector = "48850"
    
elif city_name == "산청군":
    region_button_selector = "48860"
    
elif city_name == "함양군":
    region_button_selector = "48870"

elif city_name == "거창군":
    region_button_selector = "48880"
    
elif city_name == "합천군":
    region_button_selector = "48890"

else:
    raise ValueError(f"Unknown city_name: {city_name}")

### 반복문 시작 #########################################################################################
route_inform_data[f"{city_name}"] = {}

for i, input_value in tqdm(input_values.iterrows()):
    routeno = input_value["routeno"]
    startnodenm = input_value["startnodenm"]
    print(f"    ## Start Extract:{i+1}")
    print(f"        routeno    : {routeno}")
    print(f"        startnodenm: {startnodenm}")

    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    driver = wb.Chrome(options = options)


    ## 교통카드빅데이터 종합정보시스템 -> 노선별 굴곡도 조회 url
    driver.get("https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC05&indiSel=IC0507")

    ## 클릭 함수
    def driver_clickable_processor(css_selector_value):
        click = WebDriverWait(driver, findtime).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_value))
        )    
        driver.execute_script("arguments[0].scrollIntoView();", click)
        driver.execute_script("arguments[0].click();", click)    
        
        return click

    ## 텍스트 추출 함수 
    def driver_extract_text_processor(css_selector_value):
        element = WebDriverWait(driver, findtime).until(
                        EC.presence_of_element_located((
                            By.CSS_SELECTOR, 
                            css_selector_value
                            ))
                    )
        return element.text

    ## 굴곡도 메뉴 클릭
    search_menu = WebDriverWait(driver, findtime).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#ulListIndiSel > li:nth-child(4)"))
    )
    driver.execute_script("arguments[0].scrollIntoView();", search_menu)
    driver.execute_script("arguments[0].click();", search_menu)


    ## 시도 박스 클릭
    dropdown = Select(driver.find_element(By.XPATH, '//*[@id="searchPopZoneSd"]'))
    dropdown.select_by_index(14)


    ## 지역 버튼 클릭 > 지역마다 클릭 버튼이 다름
    dropdown = Select(driver.find_element(By.XPATH, '//*[@id="searchPopZoneSgg"]'))
    dropdown.select_by_value(region_button_selector)


    ## 노선명 입력칸 클릭 -> 노선번호(명) 입력
    route_name_input = driver_clickable_processor("#popupSearchRouteNo")
    route_name_input.send_keys(f"{routeno}")


    ## 검색 버튼 클릭
    search_button = driver_clickable_processor("#route_space1 > li.box_flex > button")


    ## 노선명과 기종점이 같은지 확인
    routeno_len = int(WebDriverWait(driver, findtime).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'pop_box'))
    ).text.split()[2])


    try:
        WebDriverWait(driver, findtime).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#divBusLineList > div.pop_box > table > tbody"))
        )

        ## 검색결과 한개인 경우
        if routeno_len == 1:
            check_routeno = driver.find_element(
                By.CSS_SELECTOR, 
                "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(2)"
                ).text

            check_startnodem = driver.find_element(
                By.CSS_SELECTOR, 
                "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(4)"
                ).text.split(" - ")[0]

            ## 노선명과 기종점이 일치할 경우 체크 박스 클릭
            if check_routeno == str(routeno): ## and check_startnodem == startnodenm
                check_click = driver_clickable_processor("#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(1) > div > label > div")
                select_button = driver_clickable_processor("#popupBusLine > div:nth-child(4) > button")
                        
                ## 검색결과조회 버튼 클릭
                search_click = driver_clickable_processor("#btnSearch > button")
                            
                ## 노선 정류장수, 노선거리, 굴곡도 추출
                print("        Start Extraction1")
                
                ## 노선 정류장수
                route_len = driver_extract_text_processor("#rgrstyReportResult > table > tbody > tr:nth-child(2) > td:nth-child(3)")
                if route_len:
                    route_len = int(route_len.replace(",", ""))
                else:
                    route_len = None
                
                ## 노선거리
                route_dis = driver_extract_text_processor("#rgrstyReportResult > table > tbody > tr:nth-child(2) > td:nth-child(4)")
                if route_dis == "~":
                    route_dis = None
                else:
                    route_dis = float(route_dis.replace(",", ""))

                ## 굴곡도
                route_cra = driver_extract_text_processor("#rgrstyReportResult > table > tbody > tr:nth-child(2) > td:nth-child(5)")
                if route_cra:
                    route_cra = float(route_cra.replace(",", ""))
                else:
                    route_cra = None
                
                print("        Done Extraction1")

        ## 검색결과 여러 개인 경우
        else:         
            for j in range(1, 11):
                check_routeno = driver.find_element(
                    By.CSS_SELECTOR, 
                    f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({j}) > td:nth-child(2)"
                    ).text

                check_startnodem = driver.find_element(
                    By.CSS_SELECTOR, 
                    f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({j}) > td:nth-child(4)"
                    ).text.split(" - ")[0]

                ## 노선명과 기종점이 일치할 경우 체크 박스 클릭
                if check_routeno == str(routeno): ## and check_startnodem == startnodenm:
                    driver_clickable_processor(f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({j}) > td:nth-child(1) > div > label > div")
            
                # for-loop 중단
                break
            
            # 선택버튼 클릭
            select_button = driver_clickable_processor("#popupBusLine > div:nth-child(4) > button")
                    
            ## 검색결과조회 버튼 클릭
            search_click = driver_clickable_processor("#btnSearch > button")
            
            ## 노선 정류장수, 노선거리, 굴곡도 추출
            print("        Start Extraction")
            
            ## 노선 정류장수
            route_len = driver_extract_text_processor("#rgrstyReportResult > table > tbody > tr:nth-child(2) > td:nth-child(3)")
            if route_len:
                route_len = int(route_len.replace(",", ""))
            else:
                route_len = None
            
            ## 노선거리
            route_dis = driver_extract_text_processor("#rgrstyReportResult > table > tbody > tr:nth-child(2) > td:nth-child(4)")
            if route_dis == "~":
                route_dis = None
            else:
                route_dis = float(route_dis.replace(",", ""))

            ## 굴곡도
            route_cra = driver_extract_text_processor("#rgrstyReportResult > table > tbody > tr:nth-child(2) > td:nth-child(5)")
            if route_cra:
                route_cra = float(route_cra.replace(",", ""))
            else:
                route_cra = None
            
            print("        Done Extraction")
            
        print("        Save Usage:")
        print(f"            노선정류장수: {route_len}")
        print(f"            노선거리(km): {route_dis}")
        print(f"            굴곡도:       {route_cra}")
        
        route_inform_data[f"{city_name}"][f"{routeno}"] = {}
        route_inform_data[f"{city_name}"][f"{routeno}"]["노선정류장수"] = route_len
        route_inform_data[f"{city_name}"][f"{routeno}"]["노선거리(km)"] = route_dis
        route_inform_data[f"{city_name}"][f"{routeno}"]["굴곡도"] = route_cra
        
        
        ## 창닫기
        driver.quit()

    except:
        ## 빈칸 넣기
        route_inform_data[f"{city_name}"][f"{routeno}"] = {}
        route_inform_data[f"{city_name}"][f"{routeno}"]["노선정류장수"] = route_len
        route_inform_data[f"{city_name}"][f"{routeno}"]["노선거리(km)"] = route_dis
        route_inform_data[f"{city_name}"][f"{routeno}"]["굴곡도"] = route_cra
        
        ## 창닫기
        print("         Error")           
        driver.quit()

0it [00:00, ?it/s]

    ## Start Extract:681
        routeno    : 100
        startnodenm: 삼포교통차고지
        Start Extraction1
        Done Extraction1
        Save Usage:
        노선정류장수: 31
        노선거리(km): 8.7
        굴곡도:       6.5


1it [00:10, 10.95s/it]

    ## Start Extract:682
        routeno    : 101
        startnodenm: 삼포교통차고지
        Start Extraction1
        Done Extraction1
        Save Usage:
        노선정류장수: 42
        노선거리(km): 12.5
        굴곡도:       6.6


2it [00:24, 12.25s/it]

    ## Start Extract:683
        routeno    : 102
        startnodenm: 삼포교통차고지


2it [00:24, 12.40s/it]


KeyboardInterrupt: 

In [ ]:
route_inform_data

In [ ]:
import pickle

with open(f'../data/버스이용데이터/{city_name}_inform.pkl', 'wb') as f: 
	pickle.dump(route_inform_data, f)